In [1]:
from __future__ import annotations
import polars as pl
import inspect
import numpy as np
from typing import Tuple

In [ ]:
from dsds.metrics import roc_auc
from dsds.transform import quantile_binning, custom_binning

In [6]:
pred = np.random.random(size=1_000_000) # .reshape(-1, 1) # Predictions
actual = np.random.random(size=1_000_000) # Actuals